# After years of thought it's finally here

# This is the code that will run at runtime; every elite run of the challenger

The preprocessing code only comes into effect when mons are added to or removed from the database entirely

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
# This dictionary maps attacking types to their coverage
'''
with open(r"jsonfiles/Off.json","r") as p:
    Off=json.load(p)
print(Off)
'''
# Instead of a read/write operation, I just create the dictionary anew 3 cells later.

'\nwith open(r"jsonfiles/Off.json","r") as p:\n    Off=json.load(p)\nprint(Off)\n'

In [3]:
# This dictionary maps move names to their types

# the commented code is obsolete now that I shifted from saving just the dictionary as the json file, to saving the whole dataframe as a json file.
'''
with open(r"jsonfiles/move_dict.json","r") as p:
    move_dict=json.load(p)
print(move_dict)
'''
movetable = pd.read_json(r"jsonfiles/move_dict_table.json") 

In [4]:
move_dict=movetable['Type'].to_dict()

In [5]:
# This dataframe has everything you'd need

core_df = pd.read_json(r"jsonfiles/core_df.json") 
core_df.head()

,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,Trash,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,None,132.0,132.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Absol,Dark,None,None,150.0,115.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Accelgor,Bug,None,None,70.0,100.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,Rock,Flying,None,105.0,60.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Aggron,Steel,Rock,None,140.0,60.0,None,RU,None,Yes,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
#Type Offense dictionary
#mirror coat and counter are typeless

# This has already been loaded as a json file.

Off={           
    'Normal':[],
    'Dragon':['Dragon'],
    'Ghost':['Psychic','Ghost'],
    'Electric':['Water','Flying'],
    'Fairy':['Fighting','Dragon','Dark'],
    'Poison':['Grass','Fairy'],

    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],

    'Fire':['Grass','Ice','Bug','Steel'],
    'Bug':['Grass','Psychic','Dark'],
    
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Freeze Dry':['Water','Grass','Ground','Flying','Dragon'],


    'Psychic':['Fighting','Poison'],
    'Flying':['Grass','Fighting','Bug'],
    'Steel':['Ice','Rock','Fairy'],

    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so I added that

    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Ground':['Fire','Electric','Poison','Rock','Steel','Dragon'],    #ground is good against nearly half dragons so I added that
    
    'Dark':['Psychic','Ghost'],
    'Typeless':[],
    'nan' :[]

}

#notes: freeze dry is sneakily coded as an electric type move to include super effectiveness on water

In [7]:
core_df

,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,Trash,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,None,132.0,132.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Absol,Dark,None,None,150.0,115.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Accelgor,Bug,None,None,70.0,100.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Aerodactyl,Rock,Flying,None,105.0,60.0,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Aggron,Steel,Rock,None,140.0,60.0,None,RU,None,Yes,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,None,NaN,NaN,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Zangoose,Normal,None,None,NaN,NaN,None,RU,None,None,None,...,Payback,Pursuit,Power Trip,Counter,None,None,None,None,None,None
Zarude,Dark,Grass,None,NaN,NaN,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Zebstrika,Electric,None,None,NaN,NaN,None,RU,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [8]:
# This code has been copied from the other pokemon file

pre_coverage=core_df.drop(columns=['Tier','In Wishlist','Used','Trash'])
pre_coverage.loc[:,'Move1':]=pre_coverage.loc[:,'Move1':].apply(lambda x: x.map(move_dict), axis=1)
# pre coverage has all the move names converted to move types

coverage=pre_coverage.map(lambda x: Off.get(x))

elite_1='Water'
elite_2='Ground'
elite_3='Ice'
elite_4='Steel'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
finally_filtered =core_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons]
finally_filtered

,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,Trash,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Abomasnow,Grass,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xatu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yanmega,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,NaN,Grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
indices=np.where(finally_filtered=='Freeze Dry')
row_indices= indices[0] #col_indices = indices[1]
print(row_indices) # finds the integer index for every row which has freeze dry


freeze_dry_indexes=finally_filtered.iloc[[i for i in row_indices]].index
finally_filtered.iloc[[i for i in row_indices]]  # prints the rows corresponding to the indexes

[ 12  17  57  72  94  95 128 169 177]


,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,Trash,...,Move34,Move35,Move36,Move37,Move38,Move39,Move40,Move41,Move42,Move43
Arctozolt,Electric,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aurorus,Rock,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Delibird,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Eiscue,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glaceon,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glalie,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lapras,Water,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mr. Rime,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ninetales-Alola,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
finally_filtered.loc[freeze_dry_indexes,'Freeze_Dry_Stab']='Freeze Dry'  # creates a new column where only these rows have Freeze_Dry_Stab, the other rows have NULL.
# It's better to create a new column for freeze dry rather than adjust it into the existing stabtype columns, like the mostly unused montype3.
# Anyway we are shrinking the columns later, and I'll make sure that the max becomes 3 columns not 4 columns

In [11]:
# I have decided that using np.where is better than this following code because this will require the apply function which is slow.


# I need to account for FreezeDry somehow
# I think this code will fail if a non ice type learns freeze dry someday.

# when freeze dry appears in finally filtered, but the montype ice doesn't, that means the elite has is water but no grass, ground, flying, dragon.
# But I want this freeeze dry to be displayed to the user and also want stabscore to be increased by 1.
# maybe the other stab type of the mon is good against the elite, so I just need to add 1 to the stabscore.

#freeze_dry_mask=finally_filtered.loc['Arctozolt'].isin(['Freeze Dry']) # this applies a mask over every element of arctozolt, and replaces them with True/False
#freeze_dry_mask.any() # if one of the elements in the row is now 'True', then the final output of the row will be True.

# further coding this requires the apply function.

In [12]:
# We're processing the base typing separately from the moves because I want 3 columns reserved for the base typing.
# I don't want moves to mesh into the type columns and fill in the null spaces of the type columns.
# so I'm going to concatenate them later

useful_stab=finally_filtered.loc[:,['MonType1','MonType2','MonType3','Freeze_Dry_Stab']]       # I'm adding the Freeze Dry column here for scalability

# if the first type becomes nullified, I"m moving the second type to the spot of the first type

''' #I'm not trying to force 3 columns anymore
useful_stab = useful_stab.apply(
    lambda row: row.dropna().reset_index(drop=True).reindex(range(3)),   # a bit different from the other code because I'm trying to force 3 columns instead of letting columns shrink too much
    axis=1,
)'''

useful_stab = useful_stab.apply(

    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,

)

useful_stab=useful_stab.rename(
    columns=lambda c: f'SupereffectiveStab{c + 1}'  #scalable no matter whether we've shrunk to 2 columns or 3 columns
)
#useful_stab.columns=['SupereffectiveStab1', 'SupereffectiveStab2', 'SupereffectiveStab3'] # 4 columns might be possible if a 3 column mon gets freeze dry
useful_stab.head(20)

,SupereffectiveStab1,SupereffectiveStab2,SupereffectiveStab3
Abomasnow,Grass,Ice,NaN
Accelgor,NaN,NaN,NaN
Aerodactyl,Rock,NaN,NaN
Aggron,Steel,Rock,NaN
Alcremie,NaN,NaN,NaN
Altaria,NaN,NaN,NaN
Ambipom,NaN,NaN,NaN
Ampharos,Electric,NaN,NaN
Appletun,Grass,NaN,NaN
Arbok,NaN,NaN,NaN


In [13]:
# The stab score is how many montypes are good against the elite (how many montypes remained after the filtering and didn't get nullified.)
# the count method counts the number of non null elements.
useful_stab['StabScore']=useful_stab[[column for column in useful_stab.columns if 'SupereffectiveStab' in column]].count(axis='columns') # If I just pass in useful stab entirely instead of a list of 3 particular columns, then upon rerunning the cell it considers stabscore as a non null column. And that adds 1 to everything
useful_stab['StabScore'].loc['Aurorus']

3

In [14]:
# Now that all the freeze dry stuff is done, we should drop that freeze dry column at the end.
# Because otherwise it will be considered in the next block of code
# and the result of this is that this freeze dry column is considered as a move column.
# so aurorus already has the move freeze dry, and it also has a fake move column which has the value freeze dry.
# this is why the freeze dry column should be dropped, so that it doesn't act like a fake move.
finally_filtered=finally_filtered.drop(columns='Freeze_Dry_Stab')

In [15]:
df_where_NaNs_are_removable=finally_filtered.loc[:,'Move1':] # this ignored the stab type

df_where_NaNs_are_removable = df_where_NaNs_are_removable.apply(
    lambda row: pd.Series([item for item in row if pd.notna(item)]),
    axis=1,
)

df_where_NaNs_are_removable = df_where_NaNs_are_removable.rename(
    columns=lambda c: f'Move{c + 1}'
)
#df_where_NaNs_are_removable

create a filter later so that moves that rely on the significantly weaker stat can be removed if desired

so then that means we create a column for a pokemon, which states its dominant offense, and if no dominant offense then it's empty

my latest idea on this is to add the physical and special attack of every mon, and sort based on this.

remember to somehow let counter and mirror coat and metal burst maybe remain unfiltered as strong moves

maybe we should remember when even special attackers have moves like uturn, or physical attackers have moves like volt switch Both of these should count as supereffective good moves because the BP exceeds 60.

I should actually create a whole ass column called pivot which shows when a mon has a pivot move or not, and what this/these priority move is/are This requires me to type in uturn for a bunch of bugs too. (and volt switch/flip turn). Also need to code in parting shot, a non offensive move.

and priority moves also should be available to both, cuz the point is usually just to break sash you know what, maybe I should make a column called priority and add in all the normal priority moves like quick attack too. I probably won't add grassy glide, but I'll force add Comfey's draining moves as priority particulary for it.

do all these complicated actions with df.query

one filter should also make sure that a type is stab supereffective for a mon (but also remember that freeze dry is electric type and needs to be counted as stab)

vectorized method usually means you create another column, like df['result']=df[first] + df[second]

Also don't heavily deprioritize solar beam and stuff, cuz z moves are a thing. Maybe mention it in your application that the recommended strategy is to z move

have a filter to avoid inconsistent moves like fling, pursuit
maybe bulldoze and rock tomb too, although those moves could come under another filter

In [16]:
#final=pd.concat([finally_filtered.loc[:,:'Trash'],useful_stab,df_where_NaNs_are_removable], axis=1)

everything_before_move1=finally_filtered.loc[:,:finally_filtered.columns[(finally_filtered.columns.get_loc('Move1')-1)]] 
# finding the index of Move1, and subtracting 1 gives us the previous column.
# refer the tosplit, part_1, part_2 stuff around SrijanSort to understand the above line, and why I stopped using 'Trash' as the endpoint of slicing.


final=pd.concat([everything_before_move1,useful_stab,df_where_NaNs_are_removable], axis=1)
final

,MonType1,MonType2,MonType3,Atk,SpA,Abilities,Tier,In Wishlist,Used,Trash,...,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31
Abomasnow,Grass,Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xatu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yanmega,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Revenge,Dig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,NaN,Grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# I still need to add all those wishlist, used, tier values back
final[['Tier','In Wishlist','Used','Trash']]=core_df[['Tier','In Wishlist','Used','Trash']]
# final did have these 4 column names (in the right spots too), but they were initially null (due to .map() stuff long ago). 
# With this code, I set all the values to the values found in core_df.

In [18]:
# I've realized I also want to be able to see the full typing of the mons even if the full typing isn't useful offensively
# So at one side of the table I'll have the full typing, and at the other side I'll have the super effective typing
# so I'm directly copying the original typing too
final[['MonType1','MonType2','MonType3']]=core_df[['MonType1','MonType2','MonType3']]

I'm kinda cooked when it comes to megas. When the megas are also RU, for now, I'll give the mons the stats of their megas.
I'll also lump all abilities together, mega or not.

In [19]:
final[['Atk','SpA','Abilities']]=core_df[['Atk','SpA','Abilities']]

Code to make it sortable by stab

In [20]:
final=final.sort_values(by='StabScore', kind = 'stable', ascending=False) # The kind parameter is needed to keep the sorting stable.

In [21]:
final=final.drop(columns='StabScore') # Now that it's sorted, we no longer need this StabScore

In [22]:
# A few stuff I don't need for now, might change later
final=final.drop(columns=['Tier','In Wishlist', 'Trash'])

In [23]:
final.to_csv('Best_Mons_at_runtime_Versus_The_Elite.csv')

copy paste technician mons before and after all the move deletions so that they don't lose their moves
Also Strong Jaw mons and stuff ig

In [24]:
clean_df=clean_df.replace(to_replace=["Fling","Pursuit","Bulldoze","Rock Tomb","Dig"],value=np.NaN)

# this was the cell I used to make certain cells null

NameError: name 'clean_df' is not defined